# 올림픽에는 어떤데이터가 ?



## 분석 방향
1. 여성
2. 나이
3. 국적
4. 동계, 하계
5. 동명이인
6. 대한민국데이터
7. 종목별 금메달 나이 평균
8. 결측치의 분포 / 어느종목에서 가장많은지? // 결측치 없는 데이터만 가지고 통계분석  -> 시간대에 따른 나이, 몸무게, 키에 대한 정보
9. 1904년도의 여자 나이대의 특수성 anova
10. 참여 국가수
11. 남녀의 국가별 시간에 따른 비율변화 (추세선그리기)
12. 남녀의 시간별 키, 몸무게 변화그래프
13. 일본과의 비교 , 북한과 비교

## 1. 데이터 전처리

데이터로드

In [1]:
import pandas as pd

In [6]:
nr = pd.read_csv("noc_regions.csv")
ae = pd.read_csv("athlete_events.csv")

In [11]:
nr.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [12]:
ae.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [13]:
nr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NOC     230 non-null    object
 1   region  227 non-null    object
 2   notes   21 non-null     object
dtypes: object(3)
memory usage: 5.5+ KB


In [14]:
ae.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [15]:
nr.isnull().sum()

NOC         0
region      3
notes     209
dtype: int64

결측치는 region :3 , notes :209개 존재

데이터 상으로 region 영역에 중복데이터가 포함되어있다. 

In [30]:
len(nr['NOC'].unique())

230

noc_regions 데이터의 NOC컬럼은 230의 유니크 값을 가진다 (== 전체 row의 NOC값은 유니크 하다)

## 2.데이터 EDA

```{admonition} Q1
noc_regions.csv 데이터에서 region이 같지만 NOC 혹은 notes값이 다른 데이터들만 추출하여 duplicates_nation 변수에 저장한 후  region의 알파벳 순으로 정렬  상위 10개의 데이터를 출력하라.
```

### Ans1.

In [32]:
duplicates_nation = nr.loc[nr.region.isin(nr.loc[set(range(len(nr)))-set(list(nr[['region']].drop_duplicates().index))].region)].sort_values('region').reset_index(drop=True)

In [33]:
duplicates_nation.head(10)

,NOC,region,notes
0,ANZ,Australia,Australasia
1,AUS,Australia,NaN
2,CAN,Canada,NaN
3,NFL,Canada,Newfoundland
4,CHN,China,NaN
5,HKG,China,Hong Kong
6,CZE,Czech Republic,NaN
7,TCH,Czech Republic,NaN
8,BOH,Czech Republic,Bohemia
9,FRG,Germany,NaN


```{admonition} Q2
athlete_events.csv를 이용하여 duplicates_nation에 존재하는 NOC마다 몇 개의 데이터가 있는지 확인하라. 결과는 'duplicate_counts' 컬럼으로 만들고 상위10개데이터를 출력하라.
```

In [26]:
def count_NOC(x):
    df = ae.loc[ae.NOC ==x]
    return len(df)

duplicates_nation['duplicate_counts']  = duplicates_nation['NOC'].apply(count_NOC)

In [27]:
duplicates_nation.head(10)

,NOC,region,notes,duplicate_counts
0,ANZ,Australia,Australasia,86
1,AUS,Australia,NaN,7638
2,CAN,Canada,NaN,9733
3,NFL,Canada,Newfoundland,1
4,CHN,China,NaN,5141
5,HKG,China,Hong Kong,685
6,CZE,Czech Republic,NaN,1874
7,TCH,Czech Republic,NaN,4404
8,BOH,Czech Republic,Bohemia,153
9,FRG,Germany,NaN,3315


In [ ]:
```{admonition} Q3
athlete_events 데이터 
```

In [35]:
ae.isnull().sum()

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [36]:
ae

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


## 3.통계분석

## 4.장바구니분석

## 5.시계열 분석

## 6.예측 모델링